In [ ]:
# ! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
#hide
from fastbook import *
from fastai.vision.all import *

In [ ]:
# Downloading the Pet Image Dataset
path = untar_data(URLs.PETS)

# Setting the base path to the pet images directory we have just downloaded
Path.BASE_PATH = path

# Using the data block API
pets = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items = get_image_files,
                 splitter = RandomSplitter(seed=42),
                 get_y = using_attr(RegexLabeller(r'(.+)_\d+.jpg$'), 'name'),
                 item_tfms = Resize(460),
                 batch_tfms = aug_transforms(size=224, min_scale=0.75))

# Creating a DataLoaders object
dls = pets.dataloaders(path/'images')

# Creating a simple model
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(2)

# Dogs and Cats to Pet Breeds

## Model Interpretation
Loss functions are designed to be things computers can differentiate and optimise, not things people can understand. That's why we have metrics!
In our case, our *accuracy* is looking pretty good already, so where is our model making mistakes? We can use a confusion matrix to see where our model is doing well and where it's doing badly.

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In our case, the confusion matrix is very hard to read as we have 37x37 entries in a giant matrix.
We can instead use the `most_confused` method, which just shows the cells of the confusion matrix with the most incorrect predictions (here, with at least 5 or more).

In [ ]:
interp.most_confused(min_val=5)

## Improving Our Model

We will now look at a range of techniques to improve the training of our model and make it better.

### Learning Rate Finder
The first thing we need to set when training a model is the learning rate. It needs to be just right to train as efficiently as possible, but how do we pick a good one?

- If our learning rate is too low, it can take many, many epochs to train our model. Not only does this waste time, but it also means that we may have problems with overfitting, because every time we do a complete pass through the data, we give our model a chance to memorise it.

- If our learning rate is too high, our optimiser may have stepped too far that it totally overshot the minimum loss. Repeating that multiple times makes it get further and further away, not closer and closer!

In 2015, the researcher Leslie Smith came up with a brilliant idea, called the *learning rate finder*. The idea was to start with a very small learning rate for one mini-batch, find what the losses are afterwards, double the learning rate for another mini-batch, track the loss and so forth. This is repeated until the loss gets worse, instead of better, which is where we know we have gone too far. We then select a learning rate a bit lower than this point.

fastai's advice is to pick either:
- One order of magnitude less than where the minimum loss was achieved (i.e. minimum divided by 10)
- The last point where the loss was clearly decreasing

The learning rate finder computes these points on a curve to help us. Both of the above rules usually give around the same value.
In the first chapter, we didn't specify a learning rate but instead used the default value from the fastai librariy (which is 1e-3).

In [ ]:
lr_min, lr_steep = learn.lr_find(suggest_funcs=(minimum, steep))

In [ ]:
print(f"Minimum/10: {lr_min:.2e}, Steepest Point: {lr_steep:.2e}")

We can see that on the above plot that in the range of 1e-6 to 1e-3, nothing really happens and the model doesn't really train. Then the loss starts to decrease until it reaches a minimum, and then increase again.

We don't want a learning rate greater than 1e-1 as it will give a training that diverges, but 1e-1 is already too high as we've left the period where the loss was decreasing steadily.

In this learning rate plot, it appears that a learning rate around 3e-3 would be appropriate, so let's use that.

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(2, base_lr=3e-3)

### Unfreezing and Transfer Learning

Now that we have a good learning rate to train our model, let's look at how we can fine-tune the weights of a pre-trained model.

We now know that a Convolutional Neural Network (i.e. ImageNet) consists of many linear layers with a nonlinear activation function between each pair, followed by one ore more final linear layers with an activation function such as softmax at the very end. The final layer uses a matrix with enough columns such that the output size is the same as the number of classes in our model.

This final layer is unlikely to be useful for us when we are fine-tuning in a transfer learning setting, because it is specifically designed to classify categories in the original pre-training dataset. So when we do transfer learning, we discard it and replace it with a new linear layer with the correct number of outputs for our desired task.

This newly added linear layer will have entirely random weights, therefore our new model prior to fine-tuning has entirely random outputs. But that does not mean that it is an entirely random model! All the prior layers have been carefully trained to be good at image classification tasks in general. We want to train our new model in such a way that we allow it to remember all of these generally useful ideas from the pre-trained model, use them to solve our particular task, and only adjust them as required for the specifics of our particular task.




Our challenge when fine-tuning is to replace the random weights in our added final linear layer with weights that correctly achieve our desired task without breaking the carefully pre-trained weights and the other layers. The trick to allow this to happen is to tell the optimiser to only update the weights in those randomly added final layer but don't change the weights in the rest of the neural network at all. This is called *freezing* those pre-trained layers.

When we create a model from a pre-trained network, fastai automatically freezes all of the pre-trained layers for us. When we call the `fine_tune` method, fastai does two things:
- Trains the randomly added layers for one epoch, with all other layers frozen
- Unfreezes all of the layers, and trains them all for the number of epochs requested

Although this is a reasonable default approach, it is likely that you may get better results by doing things slightly differently. The `fine_tune` method has a number of parameters you can use to change its behaviour, but it might be easier to simply call the underlying methods directly if you want some custom behaviour.

Remember, that we can see the source code for a method by using the following syntax.

In [ ]:
learn.fine_tune??

Let's trying doing this manually ourselves!

First of all, we will train the randomly added layers for three epochs using `fit_one_cycle`.

NOTE: `fit_one_cycle` is the suggested way to train models without using `fine_tune` as it starts training at a low learning rate, gradually increasing it for the first section of training, and then gradually increasing it again for the last section of training.

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fit_one_cycle(3, 3e-3)

We'll then unfreeze the model and run `lr_find` again, because having more layers to train, and weights that have already been trained for three epochs means our previously found learning rate isn't appropriate anymore.

In [ ]:
learn.unfreeze()
learn.lr_find()

The graph is a little different from when we had random weights, we no longer have that sharp descent that indicates the model is training.
Here, we have a somewhat flat area before a sharp increase, and we should take a point well before that share increase - for instance, 1e-5.

The point with the maximum gradient isn't what we look for here and should be ignored.

Let's train the remaining layers at this suitable learning rate.

In [ ]:
learn.fit_one_cycle(6, lr_max=1e-5)

Although this has improved our model a bit, there's more we can do. 

### Discriminative Learning Rates
The deepest layers of our pre-trained model might not need a high a learning raate as the last ones, so we should probably use different learning rates for those - this is known as using *discriminative learning rates*.

Even after we unfreeze, we still care a lot about the quality of the pre-trained weights. We would not expect that the best learning rate for those pre-trained parameters would be as high as for the randomly added parameters, even after we have tuned those randomly added parameters for a few epochs. (Remember, that the pre-trained weights have been trained on hundreds of epochs)

Remembering that the initial layers learn very simple foundations, like edge and gradient detectors; these are likely to be useful for generic tasks. The layer layers learn much more complex concepts, which might not be useful in our task at all. So it makes sense to let the layers fine-tune more quickly than earlier layers.

fastai's default approach is to use *discriminative learning rates*. It uses a lower learning rate for the early layers of the neural network, and a higher learning rate for the layer layers (and especially the randomly added layers). The idea is that with transfer learning, different layers of a neural network should train at different speeds.

fastai lets us pass a Python `slice` object anywhere that a learning rate is expected. The first value passed will be the learning rate in the earliest layers of the neural network, and the second value will be the learning rate in the final layer. The layers in between will have learning rates that are multiplicatively equidistant throughout that range.

We will replicate our previous training using this approach, but we'll set the lowest layer of our neural network to a learning rate of 1e-6 whilst the other layers scale up to 1e-4.

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fit_one_cycle(3, 3e-3)
learn.unfreeze()
learn.fit_one_cycle(12, lr_max=slize(1e-6, 1e-4))

Now that the fine-tuning is working great! fastai can show us a graph of the training and validation loss.

In [ ]:
learn.recorder.plot_loss()

We can see that the training loss keeps getting better. But notice how that eventually the validation loss improvement slowws, and sometimes even gets worse. This is the point at which the model is starting to overfit.

This does *not* mean that the model is getting less accurate, necessarily. Looking at the table of training results per epoch, we often see that the accuracy continues improving, even as the validation gets worse. What matters is our chose metric, not the loss. The loss is just the function we've given the computer to help us optimise.

### Number of Epochs
Another decision to make when training a model is for how long to train for.

Often, we will be limited by time, rather than generalisation and accuracy, when choosing how many epochs to train for. So therefore, the first approach should be to simply pick a number of epochs that will train in the amount of time you are happy to wait for. Next, take a look at the training and validation plots and metrics, and if you see that they are still getting better in your final epochs, then you know you have not trained for long enough.

On the other hand, you may see that the metrics yoou have chosen are getting worse at the end of training. This may bew when the model is incorrectly memorising the data and indicates you have trained for too long.

Before *1cycle training*, it was very common to save the model at the end of each epoch, and then select whichever model had the best accuracy out of all the models saved in each epoch. This is known as *early stopping*. However, this is very unlikely to give you the best answer, because those epochs in the middle occur before the learning rate has had a chance to reach the small values, where it can really find the best result.

If you find that you have overfit, what you should actually do is to retrain your model from scratch, and this time select a total number of epochs based on where your previous best results were found.
If you have the time to train for more epochs, you may instead use that time to train for more parameters - that is, use a deeper architecture.

### Deeper Architectures
In general, a model with more parameters can model your data more accurately. For most architectures that we will be seeing in this book, we can create layer versions of them by simply adding more layers. However, since we want to use pretrained models, we need to make sure that we choose a number of layers that have been pretrained for us.

For instance, the ResNet architecture comes in variants with 18, 34, 50, 101 and 152 layers pretrained on ImageNet. A larger version of a ResNet will always be able to give us a better training loss, but it can suffer more from overfitting, because it has more parameters to overfit with. In general, a bigger model has the ability to better capture the real underlying relationships in your data, and also capture and memorise the specific details of your images.

However, using a deeper model is going to use more GPU RAM, so you may need to use a smaller batch size in your `DataLoaders`, which means passing smaller groups of images at any given time through your model.

Another downside of using deeper architectures is that they take quite a bit longer to train. One technique to speed things up is to use *mixed-precision* training. This refers to using less-precise numbers where possible during training. To enable this feature in fastai, just add `to_fp16()` after your `Learner` creation.

You can't really know ahead of time what the best architecture for your particular problem is - you need to try training some.

In [ ]:
from fastai.callback.fp16 import *
learn = vision_learner(dls, resnet50, metrics=error_rate).to_fp16()
learn.fine_tune(6, freeze_epochs=3)

We have gone back to using `fine_tune`, since it's so handy. We passed `freeze_epochs` to tell fastai how many epochs to train for while frozen. It will automatically change learning rates appropriately for most datasets.

In this case, we're not seeing a clear win from using a deeper model. So make sure to try small models before you start scaling up.